In [58]:
import urllib.request
import json
import datetime
import pandas as pd
import random
import folium

In [59]:
city = input('음식점 추천을 위한 지역구를 설정해주세요: ')

음식점 추천을 위한 지역구를 설정해주세요: 종로구


In [9]:
openweather_api_url = "https://api.openweathermap.org/data/2.5/weather?lat=37.5766296538935&lon=127.016174788877&appid=2e958182b9b690e245de1a17bf0981f8"

In [23]:
def getNowCity(city) :
    global openweather_api_url, service_key

    # API 요청시 필요한 인수값 정의
    ow_api_url = openweather_api_url
#     payload = "?q=" + city + "&" + "appid=" + service_key #"lang=kr" 옵션을 추가하면 날씨설명을 한글로 받을 수 있음.
    url_total = ow_api_url

    # API 요청하여 데이터 받기
    req = urllib.request.urlopen(url_total)
    res = req.readline()

    # 받은 값 JSON 형태로 정제하여 반환
    items = json.loads(res)
    print(items)
    print("============================")
    print("도시명 : %r" % items['name'])
    print("============================")
    print("날씨 : %r" % items['weather'][0]['main'])
    print("날씨상세 : %r" % items['weather'][0]['description'])
    print("현재온도 : %r" % str(int(items['main']['temp'])-273.15))
    print("아이콘 : %r  (사이트참조: https://openweathermap.org/weather-conditions)" % items['weather'][0]['icon'])

    print("============================")
    return items['weather'][0]['main']

In [24]:
wheather = getNowCity("Seoul")

{'coord': {'lon': 127.0162, 'lat': 37.5766}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 306.99, 'feels_like': 313.99, 'temp_min': 306.03, 'temp_max': 308.94, 'pressure': 1006, 'humidity': 58}, 'visibility': 10000, 'wind': {'speed': 4.63, 'deg': 290}, 'clouds': {'all': 75}, 'dt': 1692344522, 'sys': {'type': 1, 'id': 8096, 'country': 'KR', 'sunrise': 1692305386, 'sunset': 1692354147}, 'timezone': 32400, 'id': 1835847, 'name': 'Seoul', 'cod': 200}
도시명 : 'Seoul'
날씨 : 'Clouds'
날씨상세 : 'broken clouds'
현재온도 : '32.85000000000002'
아이콘 : '04d'  (사이트참조: https://openweathermap.org/weather-conditions)


In [25]:
wheather

'Clouds'

In [26]:
#계절기간분류
def determine_season(month):
    if 3 <= month <= 5:
        return 'spring'
    elif 6 <= month <= 8:
        return 'summer'
    elif 9 <= month <= 11:
        return 'fall'
    else:
        return 'winter'
    
#계절온도분류
def determine_season_temperature(temp):
    if temp >= 24:
        return 'summer'
    elif temp >= 8:
        return 'spring_fall'
    else:
        return 'winter'

In [27]:
current_date = datetime.datetime.now()
current_month = current_date.month #현재 달


season_by_month = determine_season(current_month)#
season_by_temp = determine_season_temperature(c_temp)#
season_by_month, season_by_temp

('summer', 'summer')

In [28]:
#조건문으로 구별
if season_by_month == season_by_temp:
    season = season_by_month
elif season_by_temp == 'spring_fall':
    if season_by_month == 'spring':
        season = 'spring'
        print('일치')
    elif season_by_month == 'fall':
        season = 'fall'
        print('일치')
    else:
        season = 'inconsistency'
        print('불일치')
else:
    season = 'inconsistency'
    print('불일치')

#계절 표기
print("현재 계절 (계절 기간 조건):", season_by_month)
print("현재 계절 (계절 온도 조건):", season_by_temp)
wheather, season

현재 계절 (계절 기간 조건): summer
현재 계절 (계절 온도 조건): summer


('Clouds', 'summer')

In [30]:
rain = {'파전': 50, '삼계탕': 30, '철판구이': 25, '중국음식': 20, '오리': 15, '곱창': 10, '쌀국수': 10,
       '햄버거': -50, '이슬람': -30, '비빔밥': -25,'냉면': -20, '족발/보쌈':-15, '카레': -10}
snow = {'삼겹살': 50, '이슬람': 30, '이태리': 25, '설렁탕': 20, '동남아': 10, '패밀리레스토랑': 15,
       '냉면': -50, '카레': -30, '한정식': -25, '곱창': -20}
spring = {'한정식': 50, '철판구이': 30, '낙지': 25, '오리': 20, '파전': 15, '불고기': 10, '비빔밥': 10, '갈비': 10}
summer = {'햄버거': 50, '삼계탕': 30, '냉면': 25, '카레': 20, '장어': 15, '전복': 10, '치킨': 10, '오리': 10}
fall = {'전라도음식': 50, '설렁탕': 30,  '이슬람': 25, '동남아': 20, '순두부': 15, '태국': 10}
winter = {'회': 50, '꽃게': 30, '쌀국수': 25, '동남아': 20, '스테이크 하우스': 15}
whole_day = {'샤브샤브': 50, '분식': 30, '족발/보쌈': 25, '삼겹살': 20, '돈까스': 15, '닭갈비': 10, '칼국수': 10, '초밥': 10, '짜장면': 10}

In [31]:
def calculate_menu(result_wheather, result_season):
    result_wheather, result_season = result_wheather.to_dict(), result_season.to_dict()
    for i in result_season.keys():
        if result_wheather.get(i):
            result_wheather[i] += result_season[i]
        else:
            result_wheather[i] = result_season[i]

    keys_to_remove = [key for key, value in result_wheather.items() if value < 0]
    for key in keys_to_remove:
        del result_wheather[key]
    
    result = sorted(result_wheather.items(), key=lambda x: x[1], reverse=True)
    print(result)
    return result

In [42]:
def wheather_menu(wheather, season):
    if wheather != 'Rain' or wheather != 'Snow':
        if season == 'inconsistency':
            result = pd.Series(whole_day)
            result = result.to_dict()
            result = sorted(result.items(), key=lambda x: x[1], reverse=True)
            print(result)
            return result
        else: #계절효과 고려 가중치
            result_wheather = pd.Series(whole_day) * 0.3
            if season == 'spring':  
                result_season = pd.Series(spring) * 0.7
            elif season == 'summer':
                result_season = pd.Series(summer) * 0.7
            elif season == 'fall': 
                result_season = pd.Series(fall) * 0.7
            else:   
                result_season = pd.Series(winter) * 0.7
            return calculate_menu(result_wheather, result_season) 
    else:
        #날씨
        if wheather == 'Rain':
            result_wheather = pd.Series(rain) * 0.7
        else:
            result_wheather = pd.Series(snow) * 0.7
        #계절
        if season == 'spring':
            result_season = pd.Series(spring) * 0.3
        elif season == 'summer': 
            result_season = pd.Series(summer) * 0.3
        elif season == 'fall':
            result_season = pd.Series(fall) * 0.3
        elif season == 'winter':
            result_season = pd.Series(winter) * 0.3
        else:
            result_season = pd.Series(whole_day) * 0.3
    
        return calculate_menu(result_wheather, result_season)

In [46]:
result = wheather_menu(wheather, season)

weights = [item[1] for item in result]
food_names = [item[0] for item in result]

random_choice = random.choices(food_names, weights=weights, k=1)[0]

[('햄버거', 35.0), ('삼계탕', 21.0), ('냉면', 17.5), ('샤브샤브', 15.0), ('카레', 14.0), ('장어', 10.5), ('분식', 9.0), ('족발/보쌈', 7.5), ('전복', 7.0), ('치킨', 7.0), ('오리', 7.0), ('삼겹살', 6.0), ('돈까스', 4.5), ('닭갈비', 3.0), ('칼국수', 3.0), ('초밥', 3.0), ('짜장면', 3.0)]


In [47]:
random_choice

'초밥'

In [45]:
(wheather, season)

('Clouds', 'summer')

In [96]:
food_data = pd.read_excel('C:/Users/user/Desktop/new_data.xlsx')

In [97]:
food_data.head()

,Unnamed: 0,도로명주소,카테고리,상호명,사이트,경도(x),위도(y),별점,지역구,음식메뉴
0,0,서울 중구 충무로2가 60-3,음식점 > 한식,명동정,http://place.map.kakao.com/26845535,126.988464,37.561785,4.40,중구,NaN
1,1,서울 중구 충무로1가 25-8,"음식점 > 한식 > 육류,고기 > 닭요리 > 봉추찜닭",봉추찜닭 명동중앙우체국점,http://place.map.kakao.com/621571654,126.982276,37.561058,4.45,중구,찜닭
2,2,서울 중구 을지로2가 199-74,"음식점 > 한식 > 육류,고기 > 닭요리 > 일미리금계찜닭",일미리금계찜닭 서울명동점,http://place.map.kakao.com/654839139,126.983656,37.565528,NaN,중구,찜닭
3,3,서울 중구 장충동2가 24-28,"음식점 > 한식 > 해물,생선 > 회",크랩회피쉬 장충동본점,http://place.map.kakao.com/32081946,127.005164,37.560665,NaN,중구,회
4,4,서울 중구 명동2가 3-3,음식점 > 한식 > 국수,곰국시집 명동점,http://place.map.kakao.com/8203856,126.985614,37.563066,NaN,중구,국수


In [98]:
result = food_data[(food_data['지역구'] == city) & (food_data['음식메뉴'] == random_choice)]

In [99]:
result.shape

(40, 10)

In [102]:
map = folium.Map(location=[37.5766296538935, 127.016174788877], zoom_start=11)

In [103]:
for n in result.index:
    folium.Marker(
        [result['위도(y)'][n], result['경도(x)'][n]],
        radius=10,
        color='#3186cc',
        fill_color='#3186cc',
        fill=True,
        tooltip=(
            '<b>- 도로명 주소</b>: ' + result['도로명주소'][n] + '<br>'
            '<b>- 상호명</b>: ' + result['상호명'][n] + '<br>' +
            '<b>- 카테고리</b>: ' + result['카테고리'][n] + '<br>' +
            '<b>- 사이트</b>: ' + result['사이트'][n] + '<br>' +
            '<b>- 네이버 평점</b>: ' + str(result['별점'][n])
        )
    ).add_to(map)
map